## Libreries

In [1]:
import pandas as pd
import pickle
import numpy as np

## Data

In [2]:
features_train = pd.read_excel("../../feature-engineering/features/lf_features_train_task_Q.xlsx", index_col="id")
features_val = pd.read_excel("../../feature-engineering/features/lf_features_val_task_Q.xlsx", index_col="id")
features_test = pd.read_excel("../../feature-engineering/features/lf_features_test_task_Q.xlsx", index_col="id")

In [4]:
Q_train = pd.read_excel("../../data/train_task_Q.xlsx", index_col="id")
Q_val = pd.read_excel("../../data/val_task_Q.xlsx", index_col="id")
Q_test = pd.read_excel("../../data/test_task_Q.xlsx", index_col="id")

In [5]:
X_train, y_train = features_train, Q_train["label"]
X_train = X_train.loc[y_train.index]

X_test, y_test = features_test, Q_test["label"]
X_test = X_test.loc[y_test.index]

X_val, y_val = features_val, Q_val["label"]
X_val = X_val.loc[y_val.index]

In [6]:
set([c.split("<&>")[0] for c in features_train.columns])

{'dep', 'is_alpha', 'is_stop', 'lemma', 'shape', 'tag'}

In [7]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [8]:
X_train.columns = range(X_train.shape[1])

In [9]:
%%time
xgb = XGBClassifier(max_depth=4, n_estimators=100, learning_rate=0.001, seed=2022)
xgb.fit(X_train, y_train)

CPU times: total: 9.48 s
Wall time: 876 ms


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.001, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=2022, reg_alpha=0, ...)

In [11]:
y_pred = xgb.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.50      0.57         4
           1       0.65      0.92      0.76        12
           2       0.85      0.85      0.85        20
           3       1.00      0.88      0.94        17
           4       1.00      0.80      0.89         5
           5       0.83      0.71      0.77         7

    accuracy                           0.83        65
   macro avg       0.83      0.78      0.80        65
weighted avg       0.85      0.83      0.83        65



In [12]:
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.12      0.07      0.09        27
           1       0.14      0.29      0.19         7
           2       0.63      0.84      0.72        32
           3       0.84      0.92      0.88       146
           4       1.00      0.50      0.67         2
           5       0.87      0.57      0.69        58

    accuracy                           0.73       272
   macro avg       0.60      0.53      0.54       272
weighted avg       0.73      0.73      0.72       272



In [13]:
from sklearn.model_selection import GridSearchCV

In [28]:
parameters = {
    'max_depth': [2, 4, 6, 8], 
    "n_estimators": [150, 125, 100],
    "learning_rate": [0.1, 0.3, 0.5, 1],
}

In [29]:
clf = GridSearchCV(xgb, parameters, scoring="f1_macro", cv=5, verbose=1)

In [30]:
%%time
clf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
CPU times: total: 35min 57s
Wall time: 3min 6s


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     callbacks=None, colsample_bylevel=1,
                                     colsample_bynode=1, colsample_bytree=1,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=0, gpu_id=-1,
                                     grow_policy='depthwise',
                                     importance_type=None,
                                     interaction_constraints='',
                                     learning_rate=0.001, max_bin=256,
                                     max_cat_to_o..., max_delta_step=0,
                                     max_depth=4, max_leaves=0,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                             

In [31]:
best_params = {}
for k in parameters:
    best_params[k] = vars(clf.best_estimator_)[k]
    print(f"{k}:", best_params[k])

max_depth: 2
n_estimators: 125
learning_rate: 1


In [32]:
y_pred = clf.predict(X_train)
report = classification_report(y_train, y_pred, output_dict=True)
train_report = pd.DataFrame(report)
train_report

,0,1,2,3,4,5,accuracy,macro avg,weighted avg
precision,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
support,13.0,48.0,78.0,64.0,17.0,27.0,1.0,247.0,247.0


In [33]:
y_pred = clf.predict(X_val)
report = classification_report(y_val, y_pred, output_dict=True)
val_report = pd.DataFrame(report)
val_report

,0,1,2,3,4,5,accuracy,macro avg,weighted avg
precision,0.666667,0.785714,0.826087,1.000000,1.00,1.000000,0.876923,0.879745,0.886415
recall,0.500000,0.916667,0.950000,0.941176,0.60,0.857143,0.876923,0.794164,0.876923
f1-score,0.571429,0.846154,0.883721,0.969697,0.75,0.923077,0.876923,0.824013,0.874006
support,4.000000,12.000000,20.000000,17.000000,5.00,7.000000,0.876923,65.000000,65.000000


In [34]:
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)
test_report = pd.DataFrame(report)
test_report

,0,1,2,3,4,5,accuracy,macro avg,weighted avg
precision,0.157895,0.058824,0.659091,0.978102,1.000000,0.945946,0.75,0.633310,0.828800
recall,0.111111,0.285714,0.906250,0.917808,0.500000,0.603448,0.75,0.554055,0.750000
f1-score,0.130435,0.097561,0.763158,0.946996,0.666667,0.736842,0.75,0.556943,0.775579
support,27.000000,7.000000,32.000000,146.000000,2.000000,58.000000,0.75,272.000000,272.000000


In [35]:
pickle.dump(clf, open("results/clf_xgboost_lf_task_Q.pickle", "wb"))

In [36]:
pickle.dump(best_params, open("results/best_params_xgboost_lf_task_Q.pickle", "wb"))

In [37]:
pickle.dump(train_report, open("results/train_report_xgboost_lf_task_Q.pickle", "wb"))

In [38]:
pickle.dump(val_report, open("results/val_report_xgboost_lf_task_Q.pickle", "wb"))

In [39]:
pickle.dump(test_report, open("results/test_report_xgboost_lf_task_Q.pickle", "wb"))